<h1 align = center> IBM Data Science Capstone Project </h1>
<h2 align = center>  The Battle of Neighborhoods </h2>
<h3 align = center > Finding the best place to open Falafel and Middle east vegan restaurant in Berlin </h3>


### Problem
In this project we will explore Berlin city to find out where the most appropriate place to open a Flafel Middle east vegan food.


### Background
Berlin is the capital and largest city of Germany by both area and population. Its 3,769,495 inhabitants as of 31 December 2019 make it the most populous city of the European Union, according to population within city limits. 
Berlin is well known for its offerings of vegetarian and vegan cuisine and is home to an innovative entrepreneurial food scene promoting cosmopolitan flavors, local and sustainable ingredients, pop-up street food markets, supper clubs, as well as food festivals, such as Berlin Food Week.
Therefore, it is one of the potential places for the success of this type of restaurant (Falafel and Middle east vegan restaurant).


### Methodology
-	Data was scraped and cleaned, and features selection was made.
-	Map created for neighborhood of Berlin
-	We use Foursquare API was used to figure out all neighborhood venues in Berlin.
-	Neighborhoods are classified using K-means unsupervised ML algorithm.


### Data
The main data sources are:
-	Wikipedia, which provide a list of districts and neighborhoods of Berlin at:
https://en.wikipedia.org/wiki/Boroughs_and_neighborhoods_of_Berlin
-	Venues data from Foursquare API
### Data acquisition and cleaning:

-	We use web scraping techniques to get data from the web (Wikipedia)
-	Convert data to Pandas dataframe, and structure all data in one table.
-	Clean data (remove unwanted columns and remove the unneeded characters from some columns).
-	Using geopy library to add latitude and longitude to each neighborhood.
-	Use Foursquare API to get venues for each neighborhood. To make it simpler we set a limit to 100 venues with radius of 1000 meters of every neighborhood.


### Importing needed libraries

In [1]:
import numpy as np # library to handle data in a vectorized manner

import pandas as pd # library for data analsysis
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)

import json # library to handle JSON files

!conda install -c conda-forge geopy --yes # uncomment this line if you haven't completed the Foursquare API lab
from geopy.geocoders import Nominatim # convert an address into latitude and longitude values

import requests # library to handle requests
from pandas.io.json import json_normalize # tranform JSON file into a pandas dataframe

# Matplotlib and associated plotting modules
import matplotlib.cm as cm
import matplotlib.colors as colors

# import k-means from clustering stage
from sklearn.cluster import KMeans

#!conda install -c conda-forge folium=0.5.0 --yes # uncomment this line if you haven't completed the Foursquare API lab
import folium # map rendering library

print('Libraries imported.')

usage: conda-script.py [-h] [-V] command ...
conda-script.py: error: unrecognized arguments: # uncomment this line if you haven't completed the Foursquare API lab


Libraries imported.


 ## Getting data
 Berlin neighborhoods data are available at \n: 
 https://en.wikipedia.org/wiki/Boroughs_and_neighborhoods_of_Berlin
 we can  scrape this data as below

In [15]:
columns = ['Locality', 'Area in km²','Population as of 2008','Density inhabitants per km²','Map']

In [55]:
df = pd.DataFrame(columns=columns)
df


,Locality,Area in km²,Population as of 2008,Density inhabitants per km²,Map


In [56]:
for i in range(2,14):
    df_temp = pd.read_html('https://en.wikipedia.org/wiki/Boroughs_and_neighborhoods_of_Berlin')[i]
    df = pd.concat([df,df_temp], ignore_index=True)

In [216]:
#df.head()

### Data cleaning

In [58]:
df['Locality'] = df['Locality'].str.extract(pat = "([a-zA-Z].*)")

In [59]:
df.drop('Map', axis =1, inplace=True)

In [63]:
df.rename(columns={'Locality':'Neighborhood'}, inplace=True)

#### Adding latitude and longtude 

In [64]:
from geopy.geocoders import Nominatim # module to convert an address into latitude and longitude values
geolocator = Nominatim(user_agent="berlin-explorer")

df['Major_Dist_Coord']= df['Neighborhood'].apply(geolocator.geocode).apply(lambda x: (x.latitude, x.longitude))
df[['Latitude', 'Longitude']] = df['Major_Dist_Coord'].apply(pd.Series)

df.drop(['Major_Dist_Coord'], axis=1, inplace=True)

In [217]:
df.head()

,Neighborhood,Area in km²,Population as of 2008,Density inhabitants per km²,Latitude,Longitude
0,Mitte,10.70,79582,7445,52.517690,13.402376
1,Moabit,7.72,69425,8993,52.530102,13.342542
2,Hansaviertel,0.53,5889,11111,52.519123,13.341872
3,Tiergarten,5.17,12486,2415,50.340922,6.956329
4,Wedding,9.23,76363,8273,52.550123,13.341970


#### Plotting Berlin map

In [66]:
address = 'Berlin'

geolocator = Nominatim(user_agent="md_explorer")
location = geolocator.geocode(address)
latitude = location.latitude
longitude = location.longitude
print('The geograpical coordinate of Berlin City are {}, {}.'.format(latitude, longitude))
print(location)

The geograpical coordinate of Berlin City are 52.5170365, 13.3888599.
Berlin, 10117, Deutschland


In [68]:
# create map of Cologne using latitude and longitude 
map_berlin = folium.Map(location=[latitude, longitude], zoom_start=11)

# add markers to the map
for lat, lng, label in zip(df['Latitude'], df['Longitude'], df['Neighborhood']):
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7,
        parse_html=False).add_to(map_berlin) 
    
map_berlin

### Foursquare credentials

In [227]:
"""
CLIENT_ID = '************************' # your Foursquare ID
CLIENT_SECRET = '*****************' # your Foursquare Secret
VERSION = '20180605' # Foursquare API version
LIMIT = 100 # A default Foursquare API limit value

print('Your credentails:')
print('CLIENT_ID: ' + CLIENT_ID)
print('CLIENT_SECRET:' + CLIENT_SECRET)
"""

"\nCLIENT_ID = '************************' # your Foursquare ID\nCLIENT_SECRET = '*****************' # your Foursquare Secret\nVERSION = '20180605' # Foursquare API version\nLIMIT = 100 # A default Foursquare API limit value\n\nprint('Your credentails:')\nprint('CLIENT_ID: ' + CLIENT_ID)\nprint('CLIENT_SECRET:' + CLIENT_SECRET)\n"

### Exploring first neighborhood

In [70]:
df.loc[0, 'Neighborhood']

'Mitte'

In [72]:
neighborhood_latitude = df.loc[0, 'Latitude'] # neighborhood latitude value
neighborhood_longitude = df.loc[0, 'Longitude'] # neighborhood longitude value

neighborhood_name = df.loc[0, 'Neighborhood'] # neighborhood name

print('Latitude and longitude values of {} are {}, {}.'.format(neighborhood_name, 
                                                               neighborhood_latitude, 
                                                               neighborhood_longitude))

Latitude and longitude values of Mitte are 52.5176896, 13.4023757.


In [73]:
LIMIT = 100

radius = 1000

url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
    CLIENT_ID, 
    CLIENT_SECRET, 
    VERSION, 
    neighborhood_latitude, 
    neighborhood_longitude, 
    radius, 
    LIMIT)
url # display URL

'https://api.foursquare.com/v2/venues/explore?&client_id=QIWXLZMWH3LRGEBKTB0LN2W5XNJTWS0DW31FNL00RFX5GRZL&client_secret=FHVDWQFHGV5XUVFQQJZKBEAZJ1LQAMSFHUDLQ34N1BRFWWND&v=20180605&ll=52.5176896,13.4023757&radius=1000&limit=100'

In [74]:
results = requests.get(url).json()

In [75]:
results

{'meta': {'code': 200, 'requestId': '5f9bc9b1dc3fa014e6af4b55'},
 'response': {'suggestedFilters': {'header': 'Tap to show:',
   'filters': [{'name': 'Open now', 'key': 'openNow'}]},
  'headerLocation': 'Museumsinsel',
  'headerFullLocation': 'Museumsinsel, Berlin',
  'headerLocationGranularity': 'neighborhood',
  'totalResults': 228,
  'suggestedBounds': {'ne': {'lat': 52.52668960900001,
    'lng': 13.417138168336054},
   'sw': {'lat': 52.508689590999985, 'lng': 13.387613231663947}},
  'groups': [{'type': 'Recommended Places',
    'name': 'recommended',
    'items': [{'reasons': {'count': 0,
       'items': [{'summary': 'This spot is popular',
         'type': 'general',
         'reasonName': 'globalInteractionReason'}]},
      'venue': {'id': '4adcda7cf964a5205f4721e3',
       'name': 'Lustgarten',
       'location': {'address': 'Am Lustgarten',
        'crossStreet': 'Schloßplatz',
        'lat': 52.518469490131785,
        'lng': 13.399454063404399,
        'labeledLatLngs': [{'la

In [76]:
# function that extracts the category of the venue
def get_category_type(row):
    try:
        categories_list = row['categories']
    except:
        categories_list = row['venue.categories']
        
    if len(categories_list) == 0:
        return None
    else:
        return categories_list[0]['name']

In [77]:
results
venues = results['response']['groups'][0]['items']
    
nearby_venues = json_normalize(venues) # flatten JSON


In [78]:
venues = results['response']['groups'][0]['items']
    
nearby_venues = json_normalize(venues) # flatten JSON

# filter columns
filtered_columns = ['venue.name', 'venue.categories', 'venue.location.lat', 'venue.location.lng']
nearby_venues =nearby_venues.loc[:, filtered_columns]

# filter the category for each row
nearby_venues['venue.categories'] = nearby_venues.apply(get_category_type, axis=1)

# clean columns
nearby_venues.columns = [col.split(".")[-1] for col in nearby_venues.columns]

nearby_venues

,name,categories,lat,lng
0,Lustgarten,Garden,52.518469,13.399454
1,Kuppelumgang Berliner Dom,Scenic Lookout,52.518966,13.400981
2,Radisson Blu,Hotel,52.519561,13.402857
3,"Bronzestatue ""Heiliger St. Georg im Kampf mit ...",Outdoor Sculpture,52.516290,13.405558
4,Designpanoptikum - surreales Museum für indust...,Museum,52.516941,13.406072
5,Deutsches Historisches Museum,History Museum,52.517788,13.396948
6,Buchhandlung Walther König,Bookstore,52.521301,13.400758
7,Altes Museum,History Museum,52.519537,13.398803
8,Nikolaiviertel,Neighborhood,52.516782,13.406453
9,Tigertörtchen,Cupcake Shop,52.517150,13.407926


## Exploring Berlin neighborhood

In [89]:
def getNearbyVenues(names, latitudes, longitudes, radius=2000):
    
    venues_list=[]
    for name, lat, lng in zip(names, latitudes, longitudes):
        print(name)
            
        # create the API request URL
        url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
            CLIENT_ID, 
            CLIENT_SECRET, 
            VERSION, 
            lat, 
            lng, 
            radius, 
            LIMIT)
            
        # make the GET request
        results = requests.get(url).json()["response"]['groups'][0]['items']
        
        # return only relevant information for each nearby venue
        venues_list.append([(
            name, 
            lat, 
            lng, 
            v['venue']['name'], 
            v['venue']['location']['lat'], 
            v['venue']['location']['lng'],  
            v['venue']['categories'][0]['name']) for v in results])

    nearby_venues = pd.DataFrame([item for venue_list in venues_list for item in venue_list])
    nearby_venues.columns = ['Neighborhood', 
                  'Neighborhood Latitude', 
                  'Neighborhood Longitude', 
                  'Venue', 
                  'Venue Latitude', 
                  'Venue Longitude', 
                  'Venue Category']
    
    return(nearby_venues)

### Getting Berlin venues

In [139]:
# type your answer here
berlin_venues = getNearbyVenues(names=df['Neighborhood'],
                                   latitudes=df['Latitude'],
                                   longitudes=df['Longitude']
                                  )

Mitte
Moabit
Hansaviertel
Tiergarten
Wedding
Gesundbrunnen
Friedrichshain
Kreuzberg
Prenzlauer Berg
Weißensee
Blankenburg
Heinersdorf
Karow
Stadtrandsiedlung Malchow
Pankow
Blankenfelde
Buch
Französisch Buchholz
Niederschönhausen
Rosenthal
Wilhelmsruh
Charlottenburg
Wilmersdorf
Schmargendorf
Grunewald
Westend
Charlottenburg-Nord
Halensee
Spandau
Haselhorst
Siemensstadt
Staaken
Gatow
Kladow
Hakenfelde
Falkenhagener Feld
Wilhelmstadt
Steglitz
Lichterfelde
Lankwitz
Zehlendorf
Dahlem
Nikolassee
Wannsee
Schöneberg
Friedenau
Tempelhof
Mariendorf
Marienfelde
Lichtenrade
Neukölln
Britz
Buckow
Rudow
Gropiusstadt
Alt-Treptow
Plänterwald
Baumschulenweg
Johannisthal
Niederschöneweide
Altglienicke
Adlershof
Bohnsdorf
Oberschöneweide
Köpenick
Friedrichshagen
Rahnsdorf
Grünau
Müggelheim
Schmöckwitz
Marzahn
Biesdorf
Kaulsdorf
Mahlsdorf
Hellersdorf
Friedrichsfelde
Karlshorst
Lichtenberg
Falkenberg
Malchow
Wartenberg
Neu-Hohenschönhausen
Alt-Hohenschönhausen
Fennpfuhl
Rummelsburg
Reinickendorf
Tegel
Kon

In [140]:
berlin_venues.shape

(5121, 7)

In [141]:
berlin_venues.groupby('Neighborhood').count()

,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
Neighborhood,,,,,,
Adlershof,45,45,45,45,45,45
Alt-Hohenschönhausen,65,65,65,65,65,65
Alt-Treptow,100,100,100,100,100,100
Altglienicke,29,29,29,29,29,29
Baumschulenweg,48,48,48,48,48,48
Biesdorf,4,4,4,4,4,4
Blankenburg,8,8,8,8,8,8
Blankenfelde,10,10,10,10,10,10
Bohnsdorf,19,19,19,19,19,19


In [142]:
print('There are {} uniques categories.'.format(len(berlin_venues['Venue Category'].unique())))

There are 361 uniques categories.


####  Analyze Each Neighborhood

In [148]:
berlin_onehot = pd.get_dummies(berlin_venues[['Venue Category']], prefix="", prefix_sep="")


In [151]:
berlin_onehot.drop('Neighborhood', axis=1, inplace=True)

In [152]:
berlin_onehot['Neighborhood'] = berlin_venues['Neighborhood'] 

In [218]:
berlin_onehot.head()

,Neighborhood,ATM,Adult Boutique,African Restaurant,Airport,Airport Lounge,Airport Service,American Restaurant,Amphitheater,Animal Shelter,Aquarium,Arcade,Argentinian Restaurant,Art Gallery,Art Museum,Arts & Crafts Store,Asian Restaurant,Athletics & Sports,Austrian Restaurant,Auto Dealership,Auto Garage,Automotive Shop,BBQ Joint,Baby Store,Bagel Shop,Bakery,Bank,Bar,Baseball Field,Bathing Area,Bavarian Restaurant,Beach,Beach Bar,Beer Bar,Beer Garden,Beer Store,Big Box Store,Bike Rental / Bike Share,Bike Shop,Bistro,Boat Rental,Boat or Ferry,Bookstore,Botanical Garden,Boutique,Bowling Alley,Brasserie,Brazilian Restaurant,Breakfast Spot,Brewery,Bridge,Bubble Tea Shop,Building,Burger Joint,Burrito Place,Bus Stop,Business Service,Butcher,Cable Car,Cafeteria,Café,Cajun / Creole Restaurant,Campground,Canal,Candy Store,Caribbean Restaurant,Carpet Store,Castle,Caucasian Restaurant,Cemetery,Cheese Shop,Chinese Restaurant,Chocolate Shop,Church,Circus,Climbing Gym,Clothing Store,Cocktail Bar,Coffee Roaster,Coffee Shop,College Cafeteria,College Gym,College Rec Center,Comedy Club,Comfort Food Restaurant,Comic Shop,Community Center,Concert Hall,Construction & Landscaping,Convenience Store,Cosmetics Shop,Creperie,Cuban Restaurant,Cupcake Shop,Currywurst Joint,Cycle Studio,Dance Studio,Deli / Bodega,Department Store,Dessert Shop,Diner,Discount Store,Distillery,Dive Bar,Dog Run,Doner Restaurant,Donut Shop,Drugstore,Dumpling Restaurant,Duty-free Shop,Eastern European Restaurant,Electronics Store,Empanada Restaurant,Escape Room,Ethiopian Restaurant,Event Space,Exhibit,Fabric Shop,Fair,Falafel Restaurant,Farm,Farmers Market,Fast Food Restaurant,Field,Fish & Chips Shop,Fish Market,Flea Market,Flower Shop,Food,Food & Drink Shop,Food Court,Food Stand,Food Truck,Football Stadium,Forest,Fountain,French Restaurant,Fried Chicken Joint,Furniture / Home Store,Garden,Garden Center,Gas Station,Gastropub,Gay Bar,General Entertainment,German Restaurant,Gift Shop,Go Kart Track,Golf Course,Gourmet Shop,Greek Restaurant,Grocery Store,Gun Range,Gym,Gym / Fitness Center,Gym Pool,Halal Restaurant,Harbor / Marina,Hardware Store,Health & Beauty Service,Historic Site,History Museum,Hockey Field,Hockey Rink,Home Service,Hookah Bar,Hostel,Hot Dog Joint,Hotel,Hotel Bar,IT Services,Ice Cream Shop,Indian Restaurant,Indie Movie Theater,Indie Theater,Indonesian Restaurant,Indoor Play Area,Industrial Estate,Insurance Office,Intersection,Irish Pub,Israeli Restaurant,Italian Restaurant,Japanese Restaurant,Jazz Club,Juice Bar,Karaoke Bar,Kebab Restaurant,Kofte Place,Korean Restaurant,Kumpir Restaurant,Kurdish Restaurant,Lake,Laser Tag,Latin American Restaurant,Laundromat,Laundry Service,Lebanese Restaurant,Library,Light Rail Station,Liquor Store,Lounge,Market,Massage Studio,Mediterranean Restaurant,Memorial Site,Men's Store,Metro Station,Mexican Restaurant,Middle Eastern Restaurant,Miscellaneous Shop,Mobile Phone Shop,Modern European Restaurant,Monument / Landmark,Motel,Motorcycle Shop,Mountain,Movie Theater,Multiplex,Museum,Music Store,Music Venue,Nail Salon,Nature Preserve,New American Restaurant,Newsstand,Nightclub,Noodle House,Nudist Beach,Opera House,Optical Shop,Organic Grocery,Outdoor Sculpture,Paintball Field,Pakistani Restaurant,Palace,Paper / Office Supplies Store,Park,Pastry Shop,Pedestrian Plaza,Performing Arts Venue,Persian Restaurant,Peruvian Restaurant,Pet Café,Pet Service,Pet Store,Pharmacy,Photography Studio,Piano Bar,Pide Place,Pie Shop,Pier,Piercing Parlor,Pizza Place,Planetarium,Platform,Playground,Plaza,Poke Place,Pool,Pool Hall,Portuguese Restaurant,Post Office,Pub,Public Art,Racetrack,Ramen Restaurant,Record Shop,Recreation Center,Rental Car Location,Rest Area,Restaurant,River,Road,Rock Climbing Spot,Rock Club,Roof Deck,Rooftop Bar,Rugby Stadium,Russian Restaurant,Salad Place,Salon / Barbershop,Sandwich Place,Sauna / Steam Room,Scandinavian Restaurant,Scenic Lookout,Schnitzel Restaurant,Science Museum,Seafood Restaurant,Shawarma Place,Shipping Store,Shoe S

In [155]:
fixed_columns = [berlin_onehot.columns[-1]] + list(berlin_onehot.columns[:-1])
berlin_onehot = berlin_onehot[fixed_columns]

berlin_onehot.head()

,Neighborhood,ATM,Adult Boutique,African Restaurant,Airport,Airport Lounge,Airport Service,American Restaurant,Amphitheater,Animal Shelter,Aquarium,Arcade,Argentinian Restaurant,Art Gallery,Art Museum,Arts & Crafts Store,Asian Restaurant,Athletics & Sports,Austrian Restaurant,Auto Dealership,Auto Garage,Automotive Shop,BBQ Joint,Baby Store,Bagel Shop,Bakery,Bank,Bar,Baseball Field,Bathing Area,Bavarian Restaurant,Beach,Beach Bar,Beer Bar,Beer Garden,Beer Store,Big Box Store,Bike Rental / Bike Share,Bike Shop,Bistro,Boat Rental,Boat or Ferry,Bookstore,Botanical Garden,Boutique,Bowling Alley,Brasserie,Brazilian Restaurant,Breakfast Spot,Brewery,Bridge,Bubble Tea Shop,Building,Burger Joint,Burrito Place,Bus Stop,Business Service,Butcher,Cable Car,Cafeteria,Café,Cajun / Creole Restaurant,Campground,Canal,Candy Store,Caribbean Restaurant,Carpet Store,Castle,Caucasian Restaurant,Cemetery,Cheese Shop,Chinese Restaurant,Chocolate Shop,Church,Circus,Climbing Gym,Clothing Store,Cocktail Bar,Coffee Roaster,Coffee Shop,College Cafeteria,College Gym,College Rec Center,Comedy Club,Comfort Food Restaurant,Comic Shop,Community Center,Concert Hall,Construction & Landscaping,Convenience Store,Cosmetics Shop,Creperie,Cuban Restaurant,Cupcake Shop,Currywurst Joint,Cycle Studio,Dance Studio,Deli / Bodega,Department Store,Dessert Shop,Diner,Discount Store,Distillery,Dive Bar,Dog Run,Doner Restaurant,Donut Shop,Drugstore,Dumpling Restaurant,Duty-free Shop,Eastern European Restaurant,Electronics Store,Empanada Restaurant,Escape Room,Ethiopian Restaurant,Event Space,Exhibit,Fabric Shop,Fair,Falafel Restaurant,Farm,Farmers Market,Fast Food Restaurant,Field,Fish & Chips Shop,Fish Market,Flea Market,Flower Shop,Food,Food & Drink Shop,Food Court,Food Stand,Food Truck,Football Stadium,Forest,Fountain,French Restaurant,Fried Chicken Joint,Furniture / Home Store,Garden,Garden Center,Gas Station,Gastropub,Gay Bar,General Entertainment,German Restaurant,Gift Shop,Go Kart Track,Golf Course,Gourmet Shop,Greek Restaurant,Grocery Store,Gun Range,Gym,Gym / Fitness Center,Gym Pool,Halal Restaurant,Harbor / Marina,Hardware Store,Health & Beauty Service,Historic Site,History Museum,Hockey Field,Hockey Rink,Home Service,Hookah Bar,Hostel,Hot Dog Joint,Hotel,Hotel Bar,IT Services,Ice Cream Shop,Indian Restaurant,Indie Movie Theater,Indie Theater,Indonesian Restaurant,Indoor Play Area,Industrial Estate,Insurance Office,Intersection,Irish Pub,Israeli Restaurant,Italian Restaurant,Japanese Restaurant,Jazz Club,Juice Bar,Karaoke Bar,Kebab Restaurant,Kofte Place,Korean Restaurant,Kumpir Restaurant,Kurdish Restaurant,Lake,Laser Tag,Latin American Restaurant,Laundromat,Laundry Service,Lebanese Restaurant,Library,Light Rail Station,Liquor Store,Lounge,Market,Massage Studio,Mediterranean Restaurant,Memorial Site,Men's Store,Metro Station,Mexican Restaurant,Middle Eastern Restaurant,Miscellaneous Shop,Mobile Phone Shop,Modern European Restaurant,Monument / Landmark,Motel,Motorcycle Shop,Mountain,Movie Theater,Multiplex,Museum,Music Store,Music Venue,Nail Salon,Nature Preserve,New American Restaurant,Newsstand,Nightclub,Noodle House,Nudist Beach,Opera House,Optical Shop,Organic Grocery,Outdoor Sculpture,Paintball Field,Pakistani Restaurant,Palace,Paper / Office Supplies Store,Park,Pastry Shop,Pedestrian Plaza,Performing Arts Venue,Persian Restaurant,Peruvian Restaurant,Pet Café,Pet Service,Pet Store,Pharmacy,Photography Studio,Piano Bar,Pide Place,Pie Shop,Pier,Piercing Parlor,Pizza Place,Planetarium,Platform,Playground,Plaza,Poke Place,Pool,Pool Hall,Portuguese Restaurant,Post Office,Pub,Public Art,Racetrack,Ramen Restaurant,Record Shop,Recreation Center,Rental Car Location,Rest Area,Restaurant,River,Road,Rock Climbing Spot,Rock Club,Roof Deck,Rooftop Bar,Rugby Stadium,Russian Restaurant,Salad Place,Salon / Barbershop,Sandwich Place,Sauna / Steam Room,Scandinavian Restaurant,Scenic Lookout,Schnitzel Restaurant,Science Museum,Seafood Restaurant,Shawarma Place,Shipping Store,Shoe S

#####  group rows by neighborhood and by taking the mean of the frequency of occurrence of each category


In [219]:
berlin_grouped = berlin_onehot.groupby('Neighborhood').mean().reset_index()
berlin_grouped

,Neighborhood,ATM,Adult Boutique,African Restaurant,Airport,Airport Lounge,Airport Service,American Restaurant,Amphitheater,Animal Shelter,Aquarium,Arcade,Argentinian Restaurant,Art Gallery,Art Museum,Arts & Crafts Store,Asian Restaurant,Athletics & Sports,Austrian Restaurant,Auto Dealership,Auto Garage,Automotive Shop,BBQ Joint,Baby Store,Bagel Shop,Bakery,Bank,Bar,Baseball Field,Bathing Area,Bavarian Restaurant,Beach,Beach Bar,Beer Bar,Beer Garden,Beer Store,Big Box Store,Bike Rental / Bike Share,Bike Shop,Bistro,Boat Rental,Boat or Ferry,Bookstore,Botanical Garden,Boutique,Bowling Alley,Brasserie,Brazilian Restaurant,Breakfast Spot,Brewery,Bridge,Bubble Tea Shop,Building,Burger Joint,Burrito Place,Bus Stop,Business Service,Butcher,Cable Car,Cafeteria,Café,Cajun / Creole Restaurant,Campground,Canal,Candy Store,Caribbean Restaurant,Carpet Store,Castle,Caucasian Restaurant,Cemetery,Cheese Shop,Chinese Restaurant,Chocolate Shop,Church,Circus,Climbing Gym,Clothing Store,Cocktail Bar,Coffee Roaster,Coffee Shop,College Cafeteria,College Gym,College Rec Center,Comedy Club,Comfort Food Restaurant,Comic Shop,Community Center,Concert Hall,Construction & Landscaping,Convenience Store,Cosmetics Shop,Creperie,Cuban Restaurant,Cupcake Shop,Currywurst Joint,Cycle Studio,Dance Studio,Deli / Bodega,Department Store,Dessert Shop,Diner,Discount Store,Distillery,Dive Bar,Dog Run,Doner Restaurant,Donut Shop,Drugstore,Dumpling Restaurant,Duty-free Shop,Eastern European Restaurant,Electronics Store,Empanada Restaurant,Escape Room,Ethiopian Restaurant,Event Space,Exhibit,Fabric Shop,Fair,Falafel Restaurant,Farm,Farmers Market,Fast Food Restaurant,Field,Fish & Chips Shop,Fish Market,Flea Market,Flower Shop,Food,Food & Drink Shop,Food Court,Food Stand,Food Truck,Football Stadium,Forest,Fountain,French Restaurant,Fried Chicken Joint,Furniture / Home Store,Garden,Garden Center,Gas Station,Gastropub,Gay Bar,General Entertainment,German Restaurant,Gift Shop,Go Kart Track,Golf Course,Gourmet Shop,Greek Restaurant,Grocery Store,Gun Range,Gym,Gym / Fitness Center,Gym Pool,Halal Restaurant,Harbor / Marina,Hardware Store,Health & Beauty Service,Historic Site,History Museum,Hockey Field,Hockey Rink,Home Service,Hookah Bar,Hostel,Hot Dog Joint,Hotel,Hotel Bar,IT Services,Ice Cream Shop,Indian Restaurant,Indie Movie Theater,Indie Theater,Indonesian Restaurant,Indoor Play Area,Industrial Estate,Insurance Office,Intersection,Irish Pub,Israeli Restaurant,Italian Restaurant,Japanese Restaurant,Jazz Club,Juice Bar,Karaoke Bar,Kebab Restaurant,Kofte Place,Korean Restaurant,Kumpir Restaurant,Kurdish Restaurant,Lake,Laser Tag,Latin American Restaurant,Laundromat,Laundry Service,Lebanese Restaurant,Library,Light Rail Station,Liquor Store,Lounge,Market,Massage Studio,Mediterranean Restaurant,Memorial Site,Men's Store,Metro Station,Mexican Restaurant,Middle Eastern Restaurant,Miscellaneous Shop,Mobile Phone Shop,Modern European Restaurant,Monument / Landmark,Motel,Motorcycle Shop,Mountain,Movie Theater,Multiplex,Museum,Music Store,Music Venue,Nail Salon,Nature Preserve,New American Restaurant,Newsstand,Nightclub,Noodle House,Nudist Beach,Opera House,Optical Shop,Organic Grocery,Outdoor Sculpture,Paintball Field,Pakistani Restaurant,Palace,Paper / Office Supplies Store,Park,Pastry Shop,Pedestrian Plaza,Performing Arts Venue,Persian Restaurant,Peruvian Restaurant,Pet Café,Pet Service,Pet Store,Pharmacy,Photography Studio,Piano Bar,Pide Place,Pie Shop,Pier,Piercing Parlor,Pizza Place,Planetarium,Platform,Playground,Plaza,Poke Place,Pool,Pool Hall,Portuguese Restaurant,Post Office,Pub,Public Art,Racetrack,Ramen Restaurant,Record Shop,Recreation Center,Rental Car Location,Rest Area,Restaurant,River,Road,Rock Climbing Spot,Rock Club,Roof Deck,Rooftop Bar,Rugby Stadium,Russian Restaurant,Salad Place,Salon / Barbershop,Sandwich Place,Sauna / Steam Room,Scandinavian Restaurant,Scenic Lookout,Schnitzel Restaurant,Science Museum,Seafood Restaurant,Shawarma Place,Shipping Store,Shoe S

In [157]:
berlin_grouped.shape

(96, 361)

In [160]:
num_top_venues = 5

for hood in berlin_grouped['Neighborhood']:
    print("----"+hood+"----")
    temp = berlin_grouped[berlin_grouped['Neighborhood'] == hood].T.reset_index()
    temp.columns = ['venue','freq']
    temp = temp.iloc[1:]
    temp['freq'] = temp['freq'].astype(float)
    temp = temp.round({'freq': 2})
    print(temp.sort_values('freq', ascending=False).reset_index(drop=True).head(num_top_venues))
    print('\n')

----Adlershof----
               venue  freq
0        Supermarket  0.16
1              Hotel  0.11
2               Café  0.09
3  German Restaurant  0.07
4          Drugstore  0.07


----Alt-Hohenschönhausen----
          venue  freq
0   Supermarket  0.12
1     Drugstore  0.08
2  Tram Station  0.08
3          Café  0.05
4        Bakery  0.03


----Alt-Treptow----
            venue  freq
0            Café  0.10
1       Nightclub  0.05
2             Bar  0.05
3  Ice Cream Shop  0.04
4          Bakery  0.04


----Altglienicke----
          venue  freq
0   Supermarket  0.24
1        Bakery  0.10
2     Drugstore  0.07
3  Liquor Store  0.07
4    Food Truck  0.07


----Baumschulenweg----
                  venue  freq
0           Supermarket  0.15
1              Bus Stop  0.08
2             Drugstore  0.08
3                  Café  0.06
4  Fast Food Restaurant  0.04


----Biesdorf----
               venue  freq
0              Hotel  0.50
1              Plaza  0.25
2  German Restaurant  0.25
3   

                venue  freq
0                Café  0.10
1         Supermarket  0.07
2   German Restaurant  0.07
3        Tram Station  0.06
4  Italian Restaurant  0.04


----Lankwitz----
             venue  freq
0      Supermarket  0.16
1        Drugstore  0.05
2           Bakery  0.05
3             Park  0.05
4  Organic Grocery  0.03


----Lichtenberg----
                venue  freq
0       Historic Site  0.25
1                Bank  0.25
2     Vacation Rental  0.25
3              Hostel  0.25
4  Photography Studio  0.00


----Lichtenrade----
                venue  freq
0         Supermarket  0.21
1            Bus Stop  0.09
2  Italian Restaurant  0.09
3              Bakery  0.06
4  Light Rail Station  0.06


----Lichterfelde----
                venue  freq
0         Supermarket  0.12
1  Italian Restaurant  0.06
2            Bus Stop  0.06
3           Drugstore  0.05
4     Organic Grocery  0.05


----Lübars----
                     venue  freq
0       Italian Restaurant  0.09
1       A

             venue  freq
0      Supermarket  0.12
1             Park  0.04
2        Drugstore  0.04
3            Plaza  0.03
4  Organic Grocery  0.03


----Westend----
                venue  freq
0      Soccer Stadium  0.06
1  Italian Restaurant  0.05
2   German Restaurant  0.05
3               Hotel  0.05
4                Café  0.05


----Wilhelmsruh----
               venue  freq
0        Supermarket  0.19
1               Park  0.06
2  German Restaurant  0.04
3              Hotel  0.04
4  Trattoria/Osteria  0.04


----Wilhelmstadt----
             venue  freq
0         Bus Stop  0.14
1      Pizza Place  0.11
2  Harbor / Marina  0.09
3      Supermarket  0.06
4             Park  0.03


----Wilmersdorf----
                venue  freq
0              Bakery  0.08
1                Café  0.06
2   German Restaurant  0.05
3               Hotel  0.05
4  Italian Restaurant  0.05


----Wittenau----
                venue  freq
0         Supermarket  0.16
1  Italian Restaurant  0.11
2     Motorcyc

#### Let's put that into a _pandas_ dataframe


In [170]:
def return_most_common_venues(row, num_top_venues):
    row_categories = row.iloc[1:]
    row_categories_sorted = row_categories.sort_values(ascending=False)
    
    return row_categories_sorted.index.values[0:num_top_venues]

In [194]:
num_top_venues = 5

indicators = ['st', 'nd', 'rd']

# create columns according to number of top venues
columns = ['Neighborhood']
for ind in np.arange(num_top_venues):
    try:
        columns.append('{}{} Most Common Venue'.format(ind+1, indicators[ind]))
    except:
        columns.append('{}th Most Common Venue'.format(ind+1))

# create a new dataframe
neighborhood_venues_sorted = pd.DataFrame(columns=columns)
neighborhood_venues_sorted['Neighborhood'] = berlin_grouped['Neighborhood']

for ind in np.arange(berlin_grouped.shape[0]):
    neighborhood_venues_sorted.iloc[ind, 1:] = return_most_common_venues(berlin_grouped.iloc[ind, :], num_top_venues)

neighborhood_venues_sorted.head()

,Neighborhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue
0,Adlershof,Supermarket,Hotel,Café,Drugstore,German Restaurant
1,Alt-Hohenschönhausen,Supermarket,Tram Station,Drugstore,Café,German Restaurant
2,Alt-Treptow,Café,Nightclub,Bar,Bakery,Coffee Shop
3,Altglienicke,Supermarket,Bakery,Drugstore,Liquor Store,Food Truck
4,Baumschulenweg,Supermarket,Drugstore,Bus Stop,Café,Furniture / Home Store


In [195]:
# set number of clusters
kclusters = 5

berlin_grouped_clustering = berlin_grouped.drop('Neighborhood', 1)

# run k-means clustering
kmeans = KMeans(n_clusters=kclusters, random_state=0).fit(berlin_grouped_clustering)

# check cluster labels generated for each row in the dataframe
kmeans.labels_[0:10] 

array([0, 0, 2, 3, 0, 4, 3, 2, 0, 0])

In [196]:
# add clustering labels
neighborhood_venues_sorted.insert(0, 'Cluster Labels', kmeans.labels_)

berlin_merged = df

# merge manhattan_grouped with manhattan_data to add latitude/longitude for each neighborhood
berlin_merged = berlin_merged.join(neighborhood_venues_sorted.set_index('Neighborhood'), on='Neighborhood')

berlin_merged.head() # check the last columns!

,Neighborhood,Area in km²,Population as of 2008,Density inhabitants per km²,Latitude,Longitude,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue
0,Mitte,10.70,79582,7445,52.517690,13.402376,2,Hotel,History Museum,Coffee Shop,Bookstore,Café
1,Moabit,7.72,69425,8993,52.530102,13.342542,2,Café,Bar,Park,Turkish Restaurant,Hotel
2,Hansaviertel,0.53,5889,11111,52.519123,13.341872,2,Hotel,Zoo Exhibit,Art Museum,Beer Garden,Gym / Fitness Center
3,Tiergarten,5.17,12486,2415,50.340922,6.956329,2,Racetrack,Hotel,German Restaurant,Rental Car Location,Sandwich Place
4,Wedding,9.23,76363,8273,52.550123,13.341970,2,Café,Bar,Park,Drugstore,Coffee Shop


In [197]:
berlin_merged.shape

(96, 12)

In [198]:
map_clusters = folium.Map(location=[latitude, longitude], zoom_start=11)

# set color scheme for the clusters
x = np.arange(kclusters)
ys = [i + x + (i*x)**2 for i in range(kclusters)]
colors_array = cm.rainbow(np.linspace(0, 1, len(ys)))
rainbow = [colors.rgb2hex(i) for i in colors_array]


In [199]:
markers_colors = []
for lat, lon, poi, cluster in zip(berlin_merged['Latitude'], berlin_merged['Longitude'], berlin_merged['Neighborhood'], berlin_merged['Cluster Labels']):
    label = folium.Popup(str(poi) + ' Cluster ' + str(cluster), parse_html=True)
    folium.CircleMarker(
        [lat, lon],
        radius=5,
        popup=label,
        color=rainbow[cluster-1],
        fill=True,
        fill_color=rainbow[cluster-1],
        fill_opacity=0.7).add_to(map_clusters)
       
map_clusters

### cluster 0

In [220]:
cluster0 = berlin_merged.loc[berlin_merged['Cluster Labels'] == 0]
cluster0

,Neighborhood,Area in km²,Population as of 2008,Density inhabitants per km²,Latitude,Longitude,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue
9,Weißensee,7.93,45485,5736,52.554619,13.463002,0,Supermarket,Drugstore,Park,Plaza,Soccer Field
13,Stadtrandsiedlung Malchow,5.68,1166,205,52.571019,13.463285,0,Pet Store,Supermarket,German Restaurant,Big Box Store,Gym / Fitness Center
18,Niederschönhausen,6.49,26903,4145,52.585806,13.401397,0,Supermarket,Café,Tram Station,Park,Greek Restaurant
28,Spandau,8.03,33433,4164,52.535788,13.197792,0,Supermarket,Pizza Place,German Restaurant,Plaza,Drugstore
29,Haselhorst,4.73,13668,2891,52.541323,13.231156,0,Supermarket,Fast Food Restaurant,Furniture / Home Store,German Restaurant,Restaurant
30,Siemensstadt,5.66,11388,2012,52.539808,13.256893,0,Supermarket,Park,Ice Cream Shop,Pizza Place,Restaurant
32,Gatow,10.10,3908,386,52.485641,13.180134,0,Bus Stop,Café,Farmers Market,Forest,Bathing Area
33,Kladow,14.80,13628,922,52.460293,13.140052,0,Supermarket,Bus Stop,Harbor / Marina,Park,Bank
34,Hakenfelde,20.40,26337,1292,52.560991,13.196769,0,Bus Stop,Supermarket,Stadium,Hotel,Harbor / Marina
35,Falkenhagener Feld,6.88,34778,5056,52.552403,13.166894,0,Bus Stop,Drugstore,Rental Car Location,Supermarket,Snack Place


### cluster 1

In [221]:
cluster1 = berlin_merged.loc[berlin_merged['Cluster Labels'] == 1]
cluster1

,Neighborhood,Area in km²,Population as of 2008,Density inhabitants per km²,Latitude,Longitude,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue
19,Rosenthal,4.9,8933,1823,50.97376,8.86942,1,Farm,Zoo Exhibit,Escape Room,Event Space,Exhibit


### cluster 2

In [222]:
cluster2 = berlin_merged.loc[berlin_merged['Cluster Labels'] == 2]
cluster2

,Neighborhood,Area in km²,Population as of 2008,Density inhabitants per km²,Latitude,Longitude,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue
0,Mitte,10.70,79582,7445,52.517690,13.402376,2,Hotel,History Museum,Coffee Shop,Bookstore,Café
1,Moabit,7.72,69425,8993,52.530102,13.342542,2,Café,Bar,Park,Turkish Restaurant,Hotel
2,Hansaviertel,0.53,5889,11111,52.519123,13.341872,2,Hotel,Zoo Exhibit,Art Museum,Beer Garden,Gym / Fitness Center
3,Tiergarten,5.17,12486,2415,50.340922,6.956329,2,Racetrack,Hotel,German Restaurant,Rental Car Location,Sandwich Place
4,Wedding,9.23,76363,8273,52.550123,13.341970,2,Café,Bar,Park,Drugstore,Coffee Shop
5,Gesundbrunnen,6.13,82729,13496,52.550920,13.384846,2,Bar,Café,Coffee Shop,Park,Pizza Place
6,Friedrichshain,9.78,114050,11662,52.512215,13.450290,2,Vegetarian / Vegan Restaurant,Café,Coffee Shop,Falafel Restaurant,Bar
7,Kreuzberg,10.40,147227,14184,52.497644,13.411914,2,Café,Coffee Shop,Bar,Turkish Restaurant,Indie Movie Theater
8,Prenzlauer Berg,11.00,142319,12991,52.539847,13.428565,2,Ice Cream Shop,Coffee Shop,Vietnamese Restaurant,Café,Beer Bar
15,Blankenfelde,13.40,1917,144,52.618627,13.388447,2,Park,Botanical Garden,Farm,Scenic Lookout,Bus Stop


### cluster 3 

In [223]:
cluster3 = berlin_merged.loc[berlin_merged['Cluster Labels'] == 3]
cluster3

,Neighborhood,Area in km²,Population as of 2008,Density inhabitants per km²,Latitude,Longitude,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue
10,Blankenburg,6.03,6550,1086,51.790268,10.955199,3,Supermarket,Mountain,Café,Gas Station,Fast Food Restaurant
11,Heinersdorf,3.95,6580,1666,52.572825,13.437015,3,Supermarket,Tram Station,Café,Hotel,Drugstore
12,Karow,6.65,18258,2746,52.615087,13.486276,3,Supermarket,Plaza,Bakery,Lake,Bus Stop
14,Pankow,5.66,55854,9868,52.597637,13.436374,3,Supermarket,Tram Station,Hotel,ATM,Asian Restaurant
16,Buch,18.20,13188,727,50.205161,7.875963,3,Liquor Store,Supermarket,Airport,Garden Center,Zoo Exhibit
17,Französisch Buchholz,12.00,18766,1560,52.610513,13.428110,3,Supermarket,Asian Restaurant,Tram Station,Garden Center,Plaza
20,Wilhelmsruh,1.37,7216,5267,52.588012,13.362206,3,Supermarket,Park,Pool,German Restaurant,Trattoria/Osteria
31,Staaken,10.90,41470,3810,52.532271,13.143367,3,Supermarket,Bus Stop,Drugstore,Fast Food Restaurant,Trattoria/Osteria
48,Marienfelde,9.15,30151,3295,52.412577,13.366592,3,Supermarket,Italian Restaurant,Bus Stop,Soccer Field,Fast Food Restaurant
49,Lichtenrade,10.10,49451,4896,52.393456,13.402040,3,Supermarket,Bus Stop,Italian Restaurant,Light Rail Station,Bakery


### cluster 4

In [225]:
cluster4 = berlin_merged.loc[berlin_merged['Cluster Labels'] == 4]
cluster4

,Neighborhood,Area in km²,Population as of 2008,Density inhabitants per km²,Latitude,Longitude,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue
67,Grünau,9.13,5482,600,-27.732656,18.378118,4,Hotel,Campground,Auto Garage,Escape Room,Event Space
71,Biesdorf,12.40,24543,1973,49.885462,6.305603,4,Hotel,German Restaurant,Plaza,Zoo Exhibit,Ethiopian Restaurant


### Results and conclusion:
 Obviously, there are three main clusters:
-	Cluster 0 with Red color on the map.
-	 cluster 2 with blue color.
-	 cluster 3 with green color

the other two clusters seem located in remote areas, and they are - definitely- not in our target.

From examining the mentioned three cluster we can note that clusters 0 and 3 are located in an industrial – companies areas, and cluster 2 is located at residential areas.
Neighborhoods that are classified as cluster 2 are seems good candidates of our target, especially that we see from our examining to this cluster that there are a few restaurants that offers Falafel and Middle east food
